### Compute Reduction in Features

In [1]:
MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_TD"

In [2]:
# for r in rows[-17:]:
#     print(str(r["asof"]))

In [6]:
import pandas as pd
import pymongo

client = pymongo.MongoClient()
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

df = pd.DataFrame(results)
# df = df.sort_values(by="f1_score",ascending=False)
cols = ("f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", "best_min_prob", 
    #"extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type")
df[list(cols)].tail(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,best_min_prob,initial_weight,loss_type,pa_type
559,0.731112,0.809439,0.666606,2019-06-16 21:33:52.602000,5,0.0005,300,1,0.0,0.01,pb,1
560,0.741985,0.785307,0.703193,2019-06-16 21:51:59.953000,5,0.0005,300,1,0.0,0.01,ml,1
561,0.707067,0.696916,0.717518,2019-06-16 22:09:41.100000,5,0.0005,300,1,0.0,0.10,pb,1
562,0.717229,0.721366,0.713139,2019-06-16 22:28:01.150000,5,0.0005,300,1,0.0,0.10,ml,1
563,0.665866,0.609348,0.733942,2019-06-16 22:48:19.551000,5,0.0005,300,1,0.0,1.00,pb,1
